In [1]:
import pandas as pd

import random
import numpy as np
%matplotlib inline
import datetime

In [2]:
is_this_one = True
if is_this_one:
    pl_df = pd.read_csv('this_one_problem_logs_seq.csv')
    ps_file = 'this_one_ps_index'
    data_file = 'this_one_train_data.csv'
else:
    pl_df = pd.read_csv('2016_problem_logs_seq.csv')
    ps_file = '2016_ps_index'
    data_file = '2016_train_data.csv'

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (21,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pl_df['formatted_start_time'] = pd.to_datetime(pl_df['start_time'])
date_before = datetime.date(2016, 7, 1)
pl_df = pl_df[pl_df['formatted_start_time'] < date_before].reset_index()

In [4]:
pl_df = pl_df[pl_df['original'] == 1].reset_index()

In [5]:
print 'The number of rows {}'.format(len(pl_df))

The number of rows 7305963


In [6]:
print 'the number of students {}'.format(len(pl_df['user_id'].unique()))

the number of students 8268


In [7]:
sub_df = pl_df
print 'the number of unique problem sets {}'.format(len(sub_df['sequence_id'].unique()))

the number of unique problem sets 164734


In [22]:
sub_counts = pd.value_counts(sub_df['sequence_id'])
minmal_cnt = 2000
ps_cnt = len(sub_counts[sub_counts > minmal_cnt])
print ps_cnt

619


In [18]:
with open(ps_file, 'w') as f:
    for ite in sub_counts[sub_counts > minmal_cnt].index.tolist():
        f.write(str(ite) + '\n')

In [11]:
sample_size = 10000
if False:
    sublist = random.sample(pl_df['user_id'].unique(), sample_size)
    sub_df = pd.DataFrame(pl_df[(pl_df['user_id'].isin(sublist))])
else:
    sub_df = pl_df
print 'the number of rows in sampled data {}'.format(len(sub_df))

the number of rows in sampled data 7305963


In [13]:
def prepare(sub_df, one_hot=True):
    new_df = pd.DataFrame()
    if one_hot:
        mask = sub_df['sequence_id'].isin(sub_counts[sub_counts > minmal_cnt].index)
        sub_df['sequence_id'][~mask] = "-"
        new_df = pd.get_dummies(sub_df['sequence_id'])
    # u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time, first_action
    new_df['user_id'] = sub_df['user_id']
    # binary correctness
    new_df['correct'] = np.where(sub_df['correct'] < 1, 0, 1)
    new_df['bottom_hint'] = sub_df['bottom_hint']
    new_df['hint_count'] = sub_df['hint_count']
    new_df['attempt_count'] = sub_df['attempt_count']
    new_df['first_response_time'] = sub_df['first_response_time']
    # first action
    new_df['first_action'] = sub_df['first_action']

    #norm = lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() - x.min() else x/(x+1)
    # normalize within problems
    #new_df.insert(len(new_df.columns), 'NofFRT', sub_df.groupby('sequence_id')['first_response_time'].transform(norm))
    new_df.insert(len(new_df.columns), 'NofFRT', sub_df.groupby('sequence_id')['first_response_time'].rank(pct=True))
    del new_df['first_response_time']

    new_df.insert(len(new_df.columns), 'normalized_hint_count', sub_df.groupby('sequence_id')['hint_count'].rank(pct=True))
    del new_df['hint_count']

    new_df.insert(len(new_df.columns), 'normalized_attempt_count', sub_df.groupby('sequence_id')['attempt_count'].rank(pct=True))
    del new_df['attempt_count']
    del new_df['user_id']
    new_df['sequence_id'] = sub_df['sequence_id']
    new_df['user_id'] = sub_df['user_id']
    new_df['id'] = sub_df['id']
    return new_df

In [14]:
new_df = prepare(sub_df, False)

In [15]:
new_df.head()

,correct,bottom_hint,first_action,NofFRT,normalized_hint_count,normalized_attempt_count,sequence_id,user_id,id
0,0,0,1,0.312193,0.999056,0.483201,5110,75064,15808280
1,0,0,1,0.600264,0.992414,0.474522,2427,75064,17558128
2,0,1,0,1.000000,0.666667,0.833333,5443,75064,18415148
3,0,0,0,0.660795,0.500284,0.484943,8728,75064,149640236
4,1,0,0,0.853977,0.500284,0.484943,8728,75064,149640353


In [16]:
new_df.to_csv(data_file, index=False)

In [17]:
new_df.isnull().values.any()

False

In [ ]:
new_df.tail()

In [ ]:
with open('this_one_all_ps_list', 'w') as file:
    for ite in new_df.columns[:ps_cnt].tolist():
        file.write(str(ite) + "\n")

### Code below is deprecated. Keep it for reference.

In [ ]:
len(pl_df.groupby('user_id').filter(lambda x: len(x) > 100))

In [ ]:

mask = pl_df['sequence_id'].isin(counts[counts > 500].index)
pl_df['sequence_id'][~mask] = "-"

new_df = pd.DataFrame()
new_df = pd.get_dummies(pl_df['sequence_id'])
# u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time
new_df['correct'] = pl_df['correct']
new_df['bottom_hint'] = pl_df['bottom_hint']
new_df['hint_count'] = pl_df['hint_count']
new_df['attempt_count'] = pl_df['attempt_count']
new_df['first_response_time'] = pl_df['first_response_time']
new_df['user_id'] = pl_df['user_id']
new_df['id'] = pl_df['id']

new_df['bottom_hint'].fillna(0, inplace=True)
norm = lambda x: (x - x.min()) / (x.max() - x.min())
new_df.insert(len(new_df.columns), 'NofFRT', pl_df.groupby('sequence_id')['first_response_time'].transform(norm))
del new_df['first_response_time']

new_df['correct'].fillna(0, inplace=True)
new_df['NofFRT'].fillna(1, inplace=True)
new_df['hint_count'].fillna(1, inplace=True)
new_df.to_csv('../LSTM-autoencoder/train_data.csv', index=False)

In [ ]:
# convert training data
train_df = pd.read_csv('22_problem_logs_seq.csv')

In [ ]:
train_df.head()

In [ ]:
# read ps_list
ps_list = []
with open('ps_list') as file:
    for line in file:
        ps_list.append(int(line))

In [ ]:
len(ps_list)

In [ ]:
encode_list = []
for index, row in train_df.iterrows():
    encode = np.zeros(len(ps_list)+1)
    try:
        idx = ps_list.index(row['sequence_id'])
        encode[idx] = 1
    except ValueError:
        encode[len(ps_list)] = 1

    encode_list.append(encode)
encode_array = np.stack(encode_list, axis=0)

In [ ]:
encode_array.shape

In [ ]:
final_df = pd.DataFrame(encode_array, columns=ps_list+['-'])

In [ ]:
final_df.tail()

In [ ]:
# u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time
final_df['correct'] = train_df['correct']
final_df['bottom_hint'] = train_df['bottom_hint']
final_df['hint_count'] = train_df['hint_count']
final_df['attempt_count'] = train_df['attempt_count']
final_df['first_response_time'] = train_df['first_response_time']

norm = lambda x: (x - x.min()) / (x.max() - x.min())
final_df.insert(len(final_df.columns), 'NofFRT', train_df.groupby('sequence_id')['first_response_time'].transform(norm))
del final_df['first_response_time']

final_df['user_id'] = train_df['user_id']
final_df['id'] = train_df['id']

In [ ]:
final_df['correct'].fillna(0, inplace=True)
final_df['NofFRT'].fillna(0.5, inplace=True)
final_df['hint_count'].fillna(1, inplace=True)
final_df['bottom_hint'].fillna(0, inplace=True)
final_df.isnull().values.any()

In [ ]:
final_df.to_csv('22_train_data.csv', index=False)